# import libraries

In [1]:
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
plt.style.use("default")
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_percentage_error,mean_absolute_error
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import keras
import pandas as pd
from utils import feature_impute, feature_impute_exiting, rf_fill, select_feature, plot_SSP
import tensorflow as tf
import tensorflow.compat.v1 as tf
import random
from sklearn.model_selection import cross_val_score, KFold
import keras.backend as K
from math import sqrt
import time
import datetime

# import dataset

In [8]:
df_all=pd.read_excel("data.xlsx")
unseen=pd.read_excel("unseen_data_fingernet.xlsx")
# df_all.columns 
Yc=['RP', 'RR']
y=df_all[Yc] 
x=df_all.drop(Yc, axis=1)
y_unseen = unseen[Yc]
x_unseen = unseen.drop(Yc, axis=1)

# data preprocessing

In [ ]:
need_delete=['Surface roughness']
need_statistic_filling=['Permeability','Rejection/Selectivity','Amine concentration','Chloride concentration']
need_model_filling=['Water contact angle','Solute concentration','NP size','Solute molecular weight']
needed_model_filling=['Amine concentration','Chloride concentration','Chloride concentration','Rejection/Selectivity']
x.drop(need_delete,axis=1,inplace=True)
x_unseen.drop(need_delete,axis=1,inplace=True)
filling=x[need_statistic_filling].mean()
x[need_statistic_filling]=x[need_statistic_filling].fillna(dict(filling), inplace=False) ##
x_unseen[need_statistic_filling]=x_unseen[need_statistic_filling].fillna(dict(filling), inplace=False) ##

In [ ]:
model=[]
for target, use_f in zip(need_model_filling,needed_model_filling):
    x, m=feature_impute(x, target, use_f)
    model.append(m)
    rf_fill(x, target)
for m, target, use_f in zip(model, need_model_filling, needed_model_filling):
    x_unseen=feature_impute_exiting(x_unseen, target, use_f, model=m)
    rf_fill(x_unseen, target)

In [11]:
df_all = pd.concat([x,x_unseen],axis=0)
col = ['Logp', 'Tpsa', 'Qed', 'Hba', 'Hbd', 'Rob']
col1 = ['Solvent_logp', 'Solvent_tpsa', 'Solvent_qed', 'Solvent_hba', 'Solvent_hbd', 'Solvent_rob']
col2 = ['Solute_logp', 'Solute_tpsa', 'Solute_qed', 'Solute_hba', 'Solute_hbd', 'Solute_rob']
smiles=pd.read_excel("smiles.xlsx")
solute = []
for i in range(df_all.shape[0]):
    if df_all.iloc[i]['Solute_type'] == 'AO':
        solute.append(smiles.loc[10][col])
    elif df_all.iloc[i]['Solute_type'] == 'BTB':
        solute.append(smiles.loc[11][col])
    elif df_all.iloc[i]['Solute_type'] == 'CV':
        solute.append(smiles.loc[12][col])
    elif df_all.iloc[i]['Solute_type'] == 'MB':
        solute.append(smiles.loc[13][col])
    elif df_all.iloc[i]['Solute_type'] == 'MO':
        solute.append(smiles.loc[14][col])
    elif df_all.iloc[i]['Solute_type'] == 'PEG':
        solute.append(smiles.loc[15][col])
    elif df_all.iloc[i]['Solute_type'] == 'RB':
        solute.append(smiles.loc[16][col])
    elif df_all.iloc[i]['Solute_type'] == 'RDB':
        solute.append(smiles.loc[17][col])
    elif df_all.iloc[i]['Solute_type'] == 'tetracycline':
        solute.append(smiles.loc[18][col])
    elif df_all.iloc[i]['Solute_type'] == 'oligomer':
        solute.append(smiles.loc[19][col])
    elif df_all.iloc[i]['Solute_type'] == 'SY':
        solute.append(smiles.loc[20][col])
    elif df_all.iloc[i]['Solute_type'] == 'EB':
        solute.append(smiles.loc[21][col])
    elif df_all.iloc[i]['Solute_type'] == 'RBBR':
        solute.append(smiles.loc[22][col])
    elif df_all.iloc[i]['Solute_type'] == 'SO':
        solute.append(smiles.loc[23][col])
df_all[col2] = solute

In [ ]:
# df_all = pd.concat([x,x_unseen],axis=0) # Combine x and x_unseen to be processed together
# col = []
# col1 = []
# col2 = []
# for i in range(1024):
#     col.append('Ecfp' + str(i))
#     col1.append('Solvent_ecfp' + str(i))
#     col2.append('Solute_ecfp' + str(i))
# solute = []
# for i in range(df_all.shape[0]):
#     if df_all.iloc[i]['Solute_type'] == 'AO':
#         solute.append(smiles.loc[10][col])
#     elif df_all.iloc[i]['Solute_type'] == 'BTB':
#         solute.append(smiles.loc[11][col])
#     elif df_all.iloc[i]['Solute_type'] == 'CV':
#         solute.append(smiles.loc[12][col])
#     elif df_all.iloc[i]['Solute_type'] == 'MB':
#         solute.append(smiles.loc[13][col])
#     elif df_all.iloc[i]['Solute_type'] == 'MO':
#         solute.append(smiles.loc[14][col])
#     elif df_all.iloc[i]['Solute_type'] == 'PEG':
#         solute.append(smiles.loc[15][col])
#     elif df_all.iloc[i]['Solute_type'] == 'RB':
#         solute.append(smiles.loc[16][col])
#     elif df_all.iloc[i]['Solute_type'] == 'RDB':
#         solute.append(smiles.loc[17][col])
#     elif df_all.iloc[i]['Solute_type'] == 'tetracycline':
#         solute.append(smiles.loc[18][col])
#     elif df_all.iloc[i]['Solute_type'] == 'oligomer':
#         solute.append(smiles.loc[19][col])
#     elif df_all.iloc[i]['Solute_type'] == 'SY':
#         solute.append(smiles.loc[20][col])
#     elif df_all.iloc[i]['Solute_type'] == 'EB':
#         solute.append(smiles.loc[21][col])
#     elif df_all.iloc[i]['Solute_type'] == 'RBBR':
#         solute.append(smiles.loc[22][col])
#     elif df_all.iloc[i]['Solute_type'] == 'SO':
#         solute.append(smiles.loc[23][col])
# df_all[col2] = solute

In [ ]:
# df_all = pd.concat([x,x_unseen],axis=0) # Combine x and x_unseen to be processed together
# col=[]
# col1=[]
# col2=[]
# for i in range(167):
#     col.append('Maccs'+str(i))
#     col1.append('Solvent_maccs'+str(i))
#     col2.append('Solute_maccs'+str(i))
#
# solute=[]
# for i in range(df_all.shape[0]):
#     if df_all.iloc[i]['Solute_type'] == 'AO':
#         solute.append(smiles.loc[10][col])
#     elif df_all.iloc[i]['Solute_type'] == 'BTB':
#         solute.append(smiles.loc[11][col])
#     elif df_all.iloc[i]['Solute_type'] == 'CV':
#         solute.append(smiles.loc[12][col])
#     elif df_all.iloc[i]['Solute_type'] == 'MB':
#         solute.append(smiles.loc[13][col])
#     elif df_all.iloc[i]['Solute_type'] == 'MO':
#         solute.append(smiles.loc[14][col])
#     elif df_all.iloc[i]['Solute_type'] == 'PEG':
#         solute.append(smiles.loc[15][col])
#     elif df_all.iloc[i]['Solute_type'] == 'RB':
#         solute.append(smiles.loc[16][col])
#     elif df_all.iloc[i]['Solute_type'] == 'RDB':
#         solute.append(smiles.loc[17][col])
#     elif df_all.iloc[i]['Solute_type'] == 'tetracycline':
#         solute.append(smiles.loc[18][col])
#     elif df_all.iloc[i]['Solute_type'] == 'oligomer':
#         solute.append(smiles.loc[19][col])
#     elif df_all.iloc[i]['Solute_type'] == 'SY':
#         solute.append(smiles.loc[20][col])
#     elif df_all.iloc[i]['Solute_type'] == 'EB':
#         solute.append(smiles.loc[21][col])
#     elif df_all.iloc[i]['Solute_type'] == 'RBBR':
#         solute.append(smiles.loc[22][col])
#     elif df_all.iloc[i]['Solute_type'] == 'SO':
#         solute.append(smiles.loc[23][col])
# df_all[col2]=solute

In [ ]:
# For data after molecular characterization of solvents and solutes,
# solvent type and solute type should be removed to prevent redundancy of information.
x=df_all.copy()
x.drop(['Solute_type'], axis=1, inplace=True) #, 'Solvent_type'
x=pd.get_dummies(x)
is_delete = []
for c in col2: # +col1
    if x[0:653][c].max() == 0:
        is_delete.append(c)
x.drop(columns=is_delete, axis=1, inplace=True)
# Use the minimum and maximum values of the former
min = x.iloc[:653,:].min()
max = x.iloc[:653,:].max()
# Make changes to the entire dataset
x = (x - min) / max
x_unseen=x.iloc[653:, :]
x=x.iloc[:653, :]
# x, x_unseen = select_feature(x, y, x_unseen)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, shuffle = True, random_state = 60)

# model building

In [13]:
def mean_squared_error(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true), axis=-1)

def my_loss(y_true, y_pred):
    scalar = 1.0*mean_squared_error(y_true[:,0],y_pred[:,0]) + 14.0*mean_squared_error(y_true[:,1], y_pred[:,1])
    return scalar
def build_model(n1=142, n2=16, lr=0.01, activation="relu", p1 = 0.1, p2 = 0.1): 
    model = keras.Sequential( 
        [ 
            keras.layers.InputLayer(input_shape = 61),
            keras.layers.Dense(units=n1, activation=activation, kernel_initializer=keras.initializers.he_normal(seed=42), bias_initializer='zeros'),
            keras.layers.Dropout(rate=p1),
            keras.layers.Dense(units=n2, activation=activation, kernel_initializer=keras.initializers.he_normal(seed=42), bias_initializer='zeros'),
            keras.layers.Dropout(rate=p2),
            keras.layers.Dense(units=2)
        ] 
    ) 
    optimizer = keras.optimizers.Adam(lr = lr) 
    model.compile(optimizer = optimizer, loss = my_loss) #my_loss
    return model 

callback1 = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                             min_delta=0,
                                             patience=30,
# The number of epochs to wait for when the evaluation metrics are not boosted, beyond which training will stop after no boosts are made
                                             verbose=0,
                                             mode='min',
                                             baseline=None,
                                             restore_best_weights=True)
callback2 = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3,
                                                 patience=30, min_lr=0.00001)

# model training

In [75]:
scores1 = []
scores2 = []
t_scores1 = []
t_scores2 = []

rmse1 = []
rmse2 = []
t_rmse1 = []
t_rmse2 = []

mape1 = []
mape2 = []
t_mape1 = []
t_mape2 = []

mae1 = []
mae2 = []
t_mae1 = []
t_mae2 = []

In [ ]:
kfold = KFold(n_splits = 5, shuffle=True, random_state=6)

for train, test in kfold.split(x_train, y_train):
    model = build_model(n1 = 338, n2 = 73, lr=0.003, p1=0.1, p2=0)
    model.fit(x_train.iloc[train], y_train.iloc[train], validation_data=(x_train.iloc[test],y_train.iloc[test]),epochs=3000, batch_size=32, verbose=0, callbacks=[callback1, callback2],shuffle=False, workers=1)
    y_train_pred=model.predict(x_train.iloc[train])
    y_val_pred=model.predict(x_train.iloc[test])
    scores1.append(r2_score(y_train.iloc[test].values[:,0],y_val_pred[:,0]))
    scores2.append(r2_score(y_train.iloc[test].values[:,1],y_val_pred[:,1]))
    t_scores1.append(r2_score(y_train.iloc[train].values[:,0],y_train_pred[:,0]))
    t_scores2.append(r2_score(y_train.iloc[train].values[:,1],y_train_pred[:,1]))
    rmse1.append(sqrt(mean_squared_error(y_train.iloc[test].values[:,0],y_val_pred[:,0])))
    rmse2.append(sqrt(mean_squared_error(y_train.iloc[test].values[:,1],y_val_pred[:,1])))
    t_rmse1.append(sqrt(mean_squared_error(y_train.iloc[train].values[:,0],y_train_pred[:,0])))
    t_rmse2.append(sqrt(mean_squared_error(y_train.iloc[train].values[:,1],y_train_pred[:,1])))
    mape1.append(mean_absolute_percentage_error(y_train.iloc[test].values[:,0],y_val_pred[:,0]))
    mape2.append(mean_absolute_percentage_error(y_train.iloc[test].values[:,1],y_val_pred[:,1]))
    t_mape1.append(mean_absolute_percentage_error(y_train.iloc[train].values[:,0],y_train_pred[:,0]))
    t_mape2.append(mean_absolute_percentage_error(y_train.iloc[train].values[:,1],y_train_pred[:,1]))
    mae1.append(mean_absolute_error(y_train.iloc[test].values[:,0],y_val_pred[:,0]))
    mae2.append(mean_absolute_error(y_train.iloc[test].values[:,1],y_val_pred[:,1]))
    t_mae1.append(mean_absolute_error(y_train.iloc[train].values[:,0],y_train_pred[:,0]))
    t_mae2.append(mean_absolute_error(y_train.iloc[train].values[:,1],y_train_pred[:,1]))
print('r2:')
print(f'train_mean1:{np.array(t_scores1).mean()}, train_std1:{np.array(t_scores1).std()}, \
        train_mean2:{np.array(t_scores2).mean()},train_std2:{np.array(t_scores2).std()},\
        test_mean1:{np.array(scores1).mean()}, test_std1:{np.array(scores1).std()},\
        test_mean2:{np.array(scores2).mean()}, test_std2:{np.array(scores2).std()}')
print('\n')
print('rmse:')
print(f'train_mean1:{np.array(t_rmse1).mean()}, train_std1:{np.array(t_rmse1).std()}, \
        train_mean2:{np.array(t_rmse2).mean()},train_std2:{np.array(t_rmse2).std()},\
        test_mean1:{np.array(rmse1).mean()}, test_std1:{np.array(rmse1).std()},\
        test_mean2:{np.array(rmse2).mean()}, test_std2:{np.array(rmse2).std()}')
print('\n')
print('mape:')
print(f'train_mean1:{np.array(t_mape1).mean()}, train_std1:{np.array(t_mape1).std()}, \
        train_mean2:{np.array(t_mape2).mean()},train_std2:{np.array(t_mape2).std()},\
        test_mean1:{np.array(mape1).mean()}, test_std1:{np.array(mape1).std()},\
        test_mean2:{np.array(mape2).mean()}, test_std2:{np.array(mape2).std()}')
print('\n')
print('mae:')
print(f'train_mean1:{np.array(t_mae1).mean()}, train_std1:{np.array(t_mae1).std()}, \
        train_mean2:{np.array(t_mae2).mean()}, train_std2:{np.array(t_mae2).std()},\
        test_mean1:{np.array(mae1).mean()}, test_std1:{np.array(mae1).std()},\
        test_mean2:{np.array(mae2).mean()}, test_std2:{np.array(mae2).std()}')

In [77]:
r2_test_rwp=[]
r2_test_rsp=[]
rmse_test_rwp=[]
rmse_test_rsp=[]
mape_test_rwp=[]
mape_test_rsp=[]
mae_test_rwp=[]
mae_test_rsp=[]
r2_train_rwp=[]
r2_train_rsp=[]
rmse_train_rwp=[]
rmse_train_rsp=[]
mape_train_rwp=[]
mape_train_rsp=[]
mae_train_rwp=[]
mae_train_rsp=[]
y_unseen_sum=np.zeros(y_unseen.shape)

In [78]:
def train(seed=6):
    tf.Session()
    random.seed(seed)
    tf.set_random_seed(seed)
    np.random.seed(seed)

    session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
    sess = tf.Session(config=session_conf)
    K.set_session(sess)
    model = build_model(n1 = 338, n2 = 73, lr=0.003, p1=0.1, p2=0)
    model.fit(x_train, y_train, validation_split=0.2, epochs=3000, batch_size=32, verbose=0, callbacks=[callback1, callback2],shuffle=False,workers=1)
    y_predt = model.predict(x_train)
    y_pred = model.predict(x_test)
    y_predu = model.predict(x_unseen)
#     print(y_predu)
    r2_test_rwp.append(r2_score(y_test.values[:, 0],y_pred[:, 0]))
    r2_test_rsp.append(r2_score(y_test.values[:, 1],y_pred[:, 1]))
    rmse_test_rwp.append(sqrt(mean_squared_error(y_test.values[:,0],y_pred[:,0])))
    rmse_test_rsp.append(sqrt(mean_squared_error(y_test.values[:,1],y_pred[:,1])))
    mape_test_rwp.append(mean_absolute_percentage_error(y_test.values[:,0],y_pred[:,0]))
    mape_test_rsp.append(mean_absolute_percentage_error(y_test.values[:,1],y_pred[:,1]))
    mae_test_rwp.append(mean_absolute_error(y_test.values[:,0],y_pred[:,0]))
    mae_test_rsp.append(mean_absolute_error(y_test.values[:,1],y_pred[:,1]))

    r2_train_rwp.append(r2_score(y_train.values[:, 0],y_predt[:, 0]))
    r2_train_rsp.append(r2_score(y_train.values[:, 1],y_predt[:, 1]))
    rmse_train_rwp.append(sqrt(mean_squared_error(y_train.values[:, 0],y_predt[:, 0])))
    rmse_train_rsp.append(sqrt(mean_squared_error(y_train.values[:, 1],y_predt[:, 1])))
    mape_train_rwp.append(mean_absolute_percentage_error(y_train.values[:, 0],y_predt[:, 0]))
    mape_train_rsp.append(mean_absolute_percentage_error(y_train.values[:, 1],y_predt[:, 1]))
    mae_train_rwp.append(mean_absolute_error(y_train.values[:, 0],y_predt[:, 0]))
    mae_train_rsp.append(mean_absolute_error(y_train.values[:, 1],y_predt[:, 1]))
#     print('R2:',r2_score(y_test.values[:, 0],y_pred[:, 0]), r2_score(y_test.values[:, 1],y_pred[:, 1]))
#     print("rmse:",sqrt(mean_squared_error(y_test.values[:,0],y_pred[:,0])), sqrt(mean_squared_error(y_test.values[:,1],y_pred[:,1])))
#     print("mape:",mean_absolute_percentage_error(y_test.values[:,0],y_pred[:,0]), mean_absolute_percentage_error(y_test.values[:,1],y_pred[:,1]))
#     print("mae:",mean_absolute_error(y_test.values[:,0],y_pred[:,0]), mean_absolute_error(y_test.values[:,1],y_pred[:,1]))
    return y_predu

In [ ]:
starttime = datetime.datetime.now()
for n, i in enumerate([random.choice([i for i in range(1000)]) for i in range(100)]):
    y_unseen_temp = train(seed=i)
    y_unseen_sum = y_unseen_sum+y_unseen_temp
    endtime = datetime.datetime.now()
    print(f'The {n+1} th randomized seed measurement, which has now taken time{(endtime - starttime).seconds} s')
endtime = datetime.datetime.now()
print(f'The total time taken for this repeat measurement：{(endtime - starttime).seconds}')

# shap summary plot

In [ ]:
# model = keras.models.load_model('***.keras', compile = False)
# plot_SSP(model, col='NP size', x_train=x_train, x_test=x_test)